In [4]:
gal_root = './gal/'
gal_file_name = 'Glint360k_r18'

In [71]:
import os

file_list = os.listdir(gal_root)

In [72]:
file_list

['a.jpg.csv', 'a.jpg.npy', 'Glint360k_r18.csv', 'Glint360k_r18.npy']

In [73]:
# 갤러리 파일명 가져오기
gal_csv = [ file for file in file_list if file.endswith('.csv') ]
gal_npy = [ file for file in file_list if file.endswith('.npy') ]
file_list = [ file for file in file_list if file.endswith('.jpg') or file.endswith('.png') ]

In [74]:
gal_csv

['a.jpg.csv', 'Glint360k_r18.csv']

In [75]:
gal_npy[0]

'a.jpg.npy'

In [78]:
import pandas as pd
import numpy as np
import torch

# 만들어진 갤러리가 있을 경우 가쟈오기만 하면 됨
gal_list = pd.read_csv(gal_root+gal_csv[0], index_col=0)
gal_embs = torch.from_numpy(np.load(gal_root+gal_npy[0]))

In [77]:
gal_list

,file,emb
0,a.jpg,True
1,b.jpg,True
2,c.jpg,True


In [79]:
gal_embs

tensor([[-13549839.0000,   9209861.0000,   8439903.0000,  ...,
           2049523.2500, -11587394.0000,  13635463.0000],
        [-12564330.0000,   7605395.5000,  12455438.0000,  ...,
          -3402834.2500,   2605661.7500,  11556930.0000],
        [-11400668.0000,   5311844.0000,    791671.2500,  ...,
         -11473002.0000, -11849430.0000,  13254015.0000]])

In [30]:
file_list = ['a.jpg','b.jpg','c.jpg']
gal_list = pd.DataFrame(file_list, columns=['file'])
gal_list['emb'] = False
gal_embs = np.zeros((3, 512), dtype=np.float32)

In [64]:
gal_embs.shape

torch.Size([3, 512])

In [ ]:
# 만들어진 갤러리가 없을 경우

In [33]:
gal_list

,file,emb
0,a.jpg,False
1,b.jpg,False
2,c.jpg,False


In [39]:
#  val = file,emb,name 으로 구성
for idx, val in gal_list.iterrows():
    face_file_name = val['file']
    lm = 1
    # image 읽어서 get_bbox()로 landmark 추출
    if lm > 0:
        face =1
        gal_embs[idx] = 512 # 얼굴 이미지 arcface에 넣어 임베딩 추출
        gal_list.loc[idx,['emb']] = True

In [41]:
gal_embs

array([[512., 512., 512., ..., 512., 512., 512.],
       [512., 512., 512., ..., 512., 512., 512.],
       [512., 512., 512., ..., 512., 512., 512.]], dtype=float32)

In [45]:
gal_list

,file,emb
0,a.jpg,True
1,b.jpg,True
2,c.jpg,True


In [48]:
# 만든 dataframe을 csv로 저장
gal_list.to_csv(gal_root+gal_file_name+'.csv')

In [49]:
np.save(gal_root+gal_file_name,gal_embs)

In [50]:
gal_embs = torch.from_numpy(gal_embs)

In [51]:
gal_embs

tensor([[512., 512., 512.,  ..., 512., 512., 512.],
        [512., 512., 512.,  ..., 512., 512., 512.],
        [512., 512., 512.,  ..., 512., 512., 512.]])

In [54]:
torch.pow(gal_embs,2).to('cuda')

tensor([[262144., 262144., 262144.,  ..., 262144., 262144., 262144.],
        [262144., 262144., 262144.,  ..., 262144., 262144., 262144.],
        [262144., 262144., 262144.,  ..., 262144., 262144., 262144.]],
       device='cuda:0')

In [ ]:
# 두 개의 임베딩 값 비교하기

In [92]:
x = gal_embs[0]
y = gal_embs
print(np.shape(x))
print(np.shape(y))


torch.Size([512])
torch.Size([3, 512])


In [88]:
x = torch.squeeze(x).to('cuda')
print(x)
print(np.shape(x))
xx = torch.dot(x, x)
print(xx)
print(np.shape(xx))

tensor([-13549839.0000,   9209861.0000,   8439903.0000,  -3847172.7500,
          8682675.0000,  16387708.0000,   6258587.5000,  11647063.0000,
          3314085.7500,  11491441.0000,   3494235.0000,  -3499212.2500,
         -5681450.5000,   5057299.0000,   2195774.7500,  -6160006.5000,
          4601739.5000,   6239390.5000,    776445.8125,   6355790.0000,
          6661636.0000,  -7272352.5000,  10472722.0000,   5096489.0000,
         -7689263.0000,  -8617092.0000,    873621.0625,  18194618.0000,
         13601701.0000,  -9347517.0000,  18693898.0000,  13900374.0000,
           504892.9375,  -4946401.0000,   2964957.2500,  14614720.0000,
         -6289592.0000,  13609773.0000,  -3211356.2500,    173888.1250,
         12066733.0000,   3441646.0000,  11239232.0000,  -7902631.0000,
        -12744655.0000,    440999.7812,   4159716.5000,   2712346.2500,
          1777747.0000,   1286079.0000, -32735964.0000,   3573805.0000,
         12236132.0000,  -1311288.2500,   -214186.1719, -1319364

tensor(3.7584e+16, device='cuda:0')
torch.Size([])


In [93]:
y = y.to('cuda')
yy = torch.pow(y, 2).to('cuda')
yy = torch.sum(yy, dim=1)
yy

tensor([3.7584e+16, 4.9584e+16, 5.0827e+16], device='cuda:0')

In [97]:
x.cuda()
y.cuda()
xy = torch.matmul(y,x)
norm = torch.sqrt(xx*yy) + 0.00001

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking arugment for argument vec in method wrapper_mv)

In [100]:
np.zeros((5,2))

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [105]:
from skimage import transform as trans
tform = trans.SimilarityTransform()
tform.params[0:2,:]

(2, 3)